<a href="https://colab.research.google.com/github/A1ienSword/Pattern-recognition-labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_10_%D0%9A%D0%BE%D1%81%D1%82%D0%B8%D1%86%D1%8B%D0%BD_%D0%92%D0%92_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: test_comp/
   creating: test_comp/F/
  inflating: test_comp/F/0026.png    
  inflating: test_comp/F/0022.png    
  inflating: test_comp/F/0020.png    
  inflating: test_comp/F/0018.png    
  inflating: test_comp/F/0003.png    
  inflating: test_comp/F/0017.png    
  inflating: test_comp/F/0019.png    
  inflating: test_comp/F/0023.png    
  inflating: test_comp/F/0014.png    
  inflating: test_comp/F/0025.png    
  inflating: test_comp/F/0000.png    
  inflating: test_comp/F/0001.png    
  inflating: test_comp/F/0011.png    
  inflating: test_comp/F/0008.png    
  inflating: test_comp/F/0010.png    
  inflating: test_comp/F/0021.png    
  inflating: test_comp/F/0002.png    
  inflating: test_comp/F/0005.png    
  inflating: test_comp/F/0027.png    
  inflating: test_comp/F/0028.png    
  inflating: test_comp/F/0016.png    
  inflating: test_comp/F/0013.png    
  inflating: test_comp/F/0029.png    
  inflating: test_comp/F/0024.png    
  inflating: tes

In [3]:
import os
import json
import numpy as np
from PIL import Image
from skimage.transform import resize
from tqdm import tqdm

In [4]:
# Глобальные настройки
TARGET_SIZE = (13, 13)
MODEL_FILE = "model.json"

def crop_image(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]] if rows.any() else (0, img.shape[0])
    xmin, xmax = np.where(cols)[0][[0, -1]] if cols.any() else (0, img.shape[1])
    return img[ymin:ymax+1, xmin:xmax+1]

def scale_image(img):
    if img.size == 0: return np.zeros(TARGET_SIZE, dtype=bool)
    scaled = resize(img, TARGET_SIZE, anti_aliasing=False)
    return scaled > 0.5

def compute_potential(img):
    potential = np.zeros(img.shape, dtype=np.float32)
    h, w = img.shape
    for y in range(h):
        for x in range(w):
            if img[y, x]:
                potential[y, x] += 1.0
                # Ближайшие соседи
                for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                    nx, ny = x+dx, y+dy
                    if 0 <= nx < w and 0 <= ny < h:
                        potential[ny, nx] += 1/6
                # Диагональные соседи
                for dx, dy in [(-1,-1), (-1,1), (1,-1), (1,1)]:
                    nx, ny = x+dx, y+dy
                    if 0 <= nx < w and 0 <= ny < h:
                        potential[ny, nx] += 1/12
    return potential

def load_image(path, preprocess=False):
    img = Image.open(path).convert('L')
    img_array = np.array(img) > 128
    if preprocess:
        cropped = crop_image(img_array)
        return scale_image(cropped)
    return img_array

def train(data_dir):
    model = {}
    classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

    # Прогресс-бар для классов
    with tqdm(total=len(classes), desc="Обработка классов") as pbar_classes:
        for class_name in classes:
            class_dir = os.path.join(data_dir, class_name)
            images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

            model[class_name] = []

            # Прогресс-бар для изображений класса
            with tqdm(total=len(images), desc=f"Класс {class_name}", leave=False) as pbar_images:
                for img_file in images:
                    img_path = os.path.join(class_dir, img_file)
                    img = load_image(img_path)
                    model[class_name].append(compute_potential(img).tolist())
                    pbar_images.update(1)

            pbar_classes.update(1)

    with open(MODEL_FILE, 'w') as f:
        json.dump(model, f)
    print(f"\nОбучение завершено. Сохранено классов: {len(model)}")


def evaluate_accuracy(test_dir):
    """Вычисляет точность модели на тестовой выборке"""
    if not os.path.exists(MODEL_FILE):
        print("Сначала обучите модель!")
        return 0.0

    # Загрузка модели
    with open(MODEL_FILE, 'r') as f:
        model = json.load(f)

    total = 0
    correct = 0

    # Прогресс-бар для классов
    classes = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]

    for true_class in tqdm(classes, desc="Оценка точности"):
        class_dir = os.path.join(test_dir, true_class)
        images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

        for img_file in images:
            img_path = os.path.join(class_dir, img_file)
            try:
                # Распознавание изображения
                predicted_class = recognize(img_path, silent=True)
                total += 1
                if predicted_class == true_class:
                    correct += 1
            except Exception as e:
                print(f"Ошибка обработки {img_path}: {str(e)}")

    accuracy = correct / total if total > 0 else 0.0
    print(f"\nТочность: {correct}/{total} ({accuracy:.2%})")
    return accuracy

def recognize(img_path, silent=False):
    """Распознает изображение и возвращает класс (silent=True отключает вывод)"""
    if not os.path.exists(MODEL_FILE):
        if not silent:
            print("Сначала обучите модель!")
        return None

    with open(MODEL_FILE, 'r') as f:
        model = json.load(f)

    img = load_image(img_path, preprocess=True)
    potential = compute_potential(img)

    min_dist = float('inf')
    best_class = "Не распознано"

    for class_name, samples in model.items():
        for sample in samples:
            sample_arr = np.array(sample)
            if sample_arr.shape != potential.shape: continue
            dist = np.sum(np.abs(sample_arr - potential))
            if dist < min_dist:
                min_dist = dist
                best_class = class_name

    if not silent:
        print(f"Результат: {best_class}")
    return best_class


In [5]:
train("train_comp")

Класс L:   0%|          | 0/66 [00:00<?, ?it/s]
                                               
Класс I:   0%|          | 0/65 [00:00<?, ?it/s]
                                               
Класс O:   0%|          | 0/68 [00:00<?, ?it/s]
                                               
Класс 10:   0%|          | 0/2 [00:00<?, ?it/s]
                                               
Класс 8:   0%|          | 0/64 [00:00<?, ?it/s]
                                               
Класс Y:   0%|          | 0/64 [00:00<?, ?it/s]
                                               
Класс E:   0%|          | 0/66 [00:00<?, ?it/s]
                                               
Класс 0:   0%|          | 0/66 [00:00<?, ?it/s]
                                               
Класс P:   0%|          | 0/66 [00:00<?, ?it/s]
                                               
Класс M:   0%|          | 0/64 [00:00<?, ?it/s]
                                               
Класс 4:   0%|          | 0/67 [00:00<?,


Обучение завершено. Сохранено классов: 37


In [6]:
evaluate_accuracy("test_comp")

Оценка точности: 100%|██████████| 37/37 [04:43<00:00,  7.67s/it]


Точность: 510/1137 (44.85%)


0.44854881266490765